# Joblib library

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_squared_error, r2_score

from joblib import dump,load

import klib
from datacleaner import autoclean


import warnings
warnings.filterwarnings('ignore')


In [17]:
df = pd.read_csv('energy_consumption.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Building Type        1000 non-null   object 
 1   Square Footage       999 non-null    float64
 2   Number of Occupants  1000 non-null   object 
 3   Appliances Used      1000 non-null   object 
 4   Average Temperature  1000 non-null   float64
 5   Day of Week          1000 non-null   object 
 6   Energy Consumption   1000 non-null   float64
dtypes: float64(3), object(4)
memory usage: 54.8+ KB


Building Type  Square Footage Number of Occupants Appliances Used  \
0   Residential          7063.0                  76              10   
1    Commercial         44372.0                  66              45   
2    Industrial         19255.0                  37              17   
3   Residential         13265.0                  14              41   
4    Commercial         13375.0                  26              18   

   Average Temperature Day of Week  Energy Consumption  
0                29.84     Weekday             2713.95  
1                16.72     Weekday             5744.99  
2                14.30     Weekend             4101.24  
3                32.82     Weekday             3009.14  
4                11.92     Weekday             3279.17

In [18]:
df = autoclean(df)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Building Type        1000 non-null   int64  
 1   Square Footage       1000 non-null   float64
 2   Number of Occupants  1000 non-null   int64  
 3   Appliances Used      1000 non-null   int64  
 4   Average Temperature  1000 non-null   float64
 5   Day of Week          1000 non-null   int64  
 6   Energy Consumption   1000 non-null   float64
dtypes: float64(3), int64(4)
memory usage: 54.8 KB


# Model training

In [21]:
x = df.drop('Energy Consumption', axis=1)
y = df['Energy Consumption']

x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [22]:
x_train.shape, x_val.shape, x_test.shape

((800, 6), (100, 6), (100, 6))

In [24]:

y_train.shape, y_val.shape, y_test.shape

((800,), (100,), (100,))

# Linear Regression

In [28]:
ln = LinearRegression()
ln.fit(x_train, y_train)
y_pred = ln.predict(x_val)


In [30]:
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)
print(f"Linear Regression MSE: {mse}")
print(f"Linear Regression R2: {r2}")

Linear Regression MSE: 195794.2473148143
Linear Regression R2: 0.792585277422619


# LGBM Regressor

In [31]:
lgb = LGBMRegressor()
lgb.fit(x_train, y_train)
y_pred = lgb.predict(x_val)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000245 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 660
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 6
[LightGBM] [Info] Start training from score 4147.958360
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

In [32]:
mse_lgb = mean_squared_error(y_val, y_pred)
r2_lgb = r2_score(y_val, y_pred)
print(f"LGBM Regressor MSE: {mse_lgb}")
print(f"LGBM Regressor R2: {r2_lgb}")

LGBM Regressor MSE: 26951.81899226893
LGBM Regressor R2: 0.9714485786180997


# Saving the model into joblib

In [33]:
dump(ln, 'ln_model.joblib')
dump(lgb, 'lgb_model.joblib')

['lgb_model.joblib']

# loading the data from joblib

In [42]:
ln = load('ln_model.joblib')
lgb = load('lgb_model.joblib')
